In [2]:
# Import everything we need
import json
import requests
import datetime
import pytz
import secretStuff #contains the notion token, the notion database ID, and the canvas token.

In [8]:
# Chose the current course we're working with
current_course_id = 1234567 # replace with the right course id
current_course_name = ""
current_course_emoji = ""

In [9]:
# Create the necessary url and headers for canvas
canvas_base = "https://canvas.uw.edu/api/v1/courses/"
canvas_assGr_URL = canvas_base + str(current_course_id) + "/assignment_groups"
canvas_header = {
    "Authorization": "Bearer " + secretStuff.canvas_token
}

# Create the necessary url and headers for notion
notion_base = "https://api.notion.com/v1/pages"
notion_headers = {
    "Authorization": "Bearer " + secretStuff.notion_token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28" # check for an update here https://developers.notion.com/reference/versioning
}

In [12]:
# Get the project groups from canvas
response = requests.get(canvas_assGr_URL, headers=canvas_header)

# Convert it into readable json format
proj_groups = response.json()

print(proj_groups)

[{'id': 1871945, 'name': 'Deliverables', 'position': 1, 'group_weight': 0.0, 'sis_source_id': None, 'integration_data': {}, 'rules': {}}, {'id': 1871946, 'name': 'Check-ins', 'position': 2, 'group_weight': 0.0, 'sis_source_id': None, 'integration_data': {}, 'rules': {}}, {'id': 1871947, 'name': '1:1 Deep Dives', 'position': 3, 'group_weight': 0.0, 'sis_source_id': None, 'integration_data': {}, 'rules': {}}, {'id': 1871948, 'name': 'Resource Summaries', 'position': 4, 'group_weight': 0.0, 'sis_source_id': None, 'integration_data': {}, 'rules': {}}]


In [6]:
#  For each project group get all the assignments, clean up the data, and put it in a page
for i in range(0, len(proj_groups)):
    # get the id for the assignment group we're currently looking at
    assGrID = str(proj_groups[i]["id"])

    # make the URL to get the assignments in that group
    canvas_ass_URL = canvas_assGr_URL + "/" + assGrID + "/assignments"

    print(canvas_ass_URL)

    # get the assignments in that project group from canvas
    response = requests.get(canvas_ass_URL, headers=canvas_header)

    # Convert it into readable json format
    assignments = response.json()

    # For each assignment clean up the data and pop it into canvas
    for k in range(len(assignments)):

        # Pop off the assignment we need right now
        current = assignments[k]
        
        # Format part of the assignment URL we need to make it accessible as a UW student
        better_URL = current["html_url"].replace("https://canvas.instructure.com", "https://canvas.uw.edu")

        # Format the date to be equipped to timezone change
        if (current["due_at"] is None):
            better_date = ""
        else:
            # First make the assignment due date into a datetime object
            dt = datetime.datetime.strptime(current["due_at"], "%Y-%m-%dT%H:%M:%SZ")
            # Then make sure the timezone of the datetime object is UTC
            dt_UTC = pytz.utc.localize(dt)
            # Then switch the timezone to where you are
            dt_pst = dt_UTC.astimezone(pytz.timezone("America/Los_Angeles")) 
            # Then make sure the date we put into the page is in ISO format
            better_date = dt_pst.isoformat()

        # Create a new dictionary that will populate the new notion page of this assignment
        pageData = {
            "parent": { 
                "database_id": secretStuff.notion_database_id
            },
            "icon": {
                "type": "emoji",
                "emoji": "🎓"
            },
            "properties": {
                "Title": {
                    "title": [
                        {
                            "text": {
                                "content": current["name"]
                            }
                        }
                    ]
                },
                "Due Date": {
                    "date": {
                        "start": better_date
                    }
                },
                "URL" : {
                    "url": better_URL
                },
                "Class": {
                    "select": {
                        "name": current_course_name
                    }
                }
            }
        }

        # Convert the new dictionary pageData to json for sending
        data = json.dumps(pageData)

        # Send the new page out to notion
        res = requests.request("POST", notion_base, headers=notion_headers, data=data)

        # Troubleshoot helpers
        print(res.status_code)
        print(res.text)


https://canvas.uw.edu/api/v1/courses/1634414/assignment_groups/1871945/assignments
200
{"object":"page","id":"84089d4d-1ca4-44e8-872b-1f90278e3afe","created_time":"2023-03-28T20:08:00.000Z","last_edited_time":"2023-03-28T20:08:00.000Z","created_by":{"object":"user","id":"e0450558-0de3-46ed-879e-02d5a5be735f"},"last_edited_by":{"object":"user","id":"e0450558-0de3-46ed-879e-02d5a5be735f"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"c856c40d-ce47-4c64-bad5-95748a1774ff"},"archived":false,"properties":{"📜 User Stories":{"id":"%3D~v%3A","type":"relation","relation":[],"has_more":false},"Sub-item":{"id":"I%3C%7Cv","type":"relation","relation":[],"has_more":false},"Parent item":{"id":"Jv%3Dc","type":"relation","relation":[],"has_more":false},"Status":{"id":"%5C%5ERl","type":"status","status":{"id":"aa804e92-cebb-49c6-8181-e186a7c6ad64","name":"Not started","color":"default"}},"Who's assigned to the user story":{"id":"bWgt","type":"rollup","rollup":{"type":"array","a